In [22]:
# Import standard library modules
import sys

# Set the relative path to the project root directory
relative_path_to_root = "../../"

# Add the project root to the system path for importing in-house modules
sys.path.append(relative_path_to_root)

# Import in-house modules from the 'utilities' package
from utilities import clean_historical_data, check_tickers_for_missing_values
from utilities import calc_vif, calc_p_values, calc_correlation, highlight_vif, highlight_p_values, evaluate_regression_model, evaluate_cross_validation, evaluate_classifier_model
from utilities import load_data, save_data
from utilities import calculate_bollinger_bands, calculate_rsi, calculate_daily_volatility
from utilities import print_title, print_label, print_footer
from utilities import generate_trading_signals
from utilities import fetch_and_download_sp500_data, sp500_data_for_today
from utilities import print_dataframe_report, save_data
from utilities import temporal_train_test_split, split_dataset_by_date

In [19]:
# Data manipulation and analysis
import pandas as pd

# Date and time manipulation
from datetime import date

# File and directory manipulation
from pathlib import Path

# Data preprocessing and model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [20]:
file_name = "updated_w_nas"
file_path = f"../../data/raw_data/{file_name}"

raw_data = load_data(file_path)

╔═══════════════════════════════════════════════════════════════╗
║ File `updated_w_nas.csv.bz2` loaded from `updated_w_nas.zip`  ║
╚═══════════════════════════════════════════════════════════════╝


In [21]:
raw_data

,Date,Ticker,Adjusted Close,Today to Tomorrow,Yesterday to Today,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2008-01-02,A,23.256388,-1.0,-1.0,23.025745,23.538284,-0.011976,0.015704,48.827595,23.314173,23.299887,23.564934,24.727250,22.540232,21.392027,24.351938,short
1,2008-01-02,AAPL,5.876342,1.0,-1.0,5.879056,5.974059,-0.016357,0.018937,59.067417,5.518483,4.939064,4.197630,6.135834,5.403559,4.637375,6.026838,buy
2,2008-01-02,ABBV,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
3,2008-01-02,ABNB,0.000000,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,2008-01-02,ABT,18.130213,-1.0,-1.0,18.019760,18.240650,-0.006054,0.010484,34.677270,18.138458,17.628250,17.709028,19.233109,18.221802,16.775555,19.134012,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121730,2024-10-28,XYL,130.835000,NaN,1.0,NaN,130.420000,0.003182,0.009305,39.970257,133.478420,134.334370,130.949620,138.418910,129.503590,126.710000,137.530000,NaN
2121731,2024-10-28,YUM,134.870000,NaN,1.0,NaN,133.040000,0.013755,0.006897,50.767624,134.574110,133.394600,134.047180,138.692950,130.995060,129.710000,139.920000,NaN
2121732,2024-10-28,ZBH,103.550000,NaN,1.0,NaN,102.350000,0.011725,0.010933,56.671688,107.229390,107.883995,115.284256,107.086190,101.507810,101.770000,115.912370,NaN
2121733,2024-10-28,ZBRA,363.530000,NaN,1.0,NaN,360.090000,0.009553,0.009969,43.505260,356.281000,337.452000,311.584440,380.047030,359.120970,320.770000,377.680000,NaN


In [28]:
todays_date = "2024-10-28"

historical_data, todays_data = split_dataset_by_date(raw_data, todays_date)

print("Todays Date:", todays_date)

Todays Date: 2024-10-28


In [29]:
todays_data

,Date,Ticker,Adjusted Close,Today to Tomorrow,Yesterday to Today,Next Day Close,Previous Day Close,Return,Volatility,RSI,SMA_50,SMA_100,SMA_200,Upper Band,Lower Band,Support,Resistance,Action
0,2024-10-28,A,131.620,NaN,1.0,NaN,130.19,0.010984,0.012264,21.425411,140.169080,136.640840,137.961560,151.502270,128.73973,130.1900,148.24400,NaN
1,2024-10-28,AAPL,233.490,NaN,1.0,NaN,231.41,0.008988,0.013784,63.955185,226.950400,222.175250,201.011900,237.876570,222.29343,216.3200,236.48000,NaN
2,2024-10-28,ABBV,189.740,NaN,1.0,NaN,187.85,0.010061,0.008103,45.739376,192.653440,183.300860,174.407040,196.414630,185.96626,186.5400,197.77069,NaN
3,2024-10-28,ABNB,135.820,NaN,1.0,NaN,134.58,0.009214,0.014882,57.457336,125.337400,133.320100,143.597980,140.080760,125.49424,114.2800,137.19000,NaN
4,2024-10-28,ABT,114.130,NaN,-1.0,NaN,114.22,-0.000788,0.010339,50.131912,114.222595,109.517050,109.521515,119.484985,111.09392,110.2504,119.39000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,2024-10-28,XYL,130.835,NaN,1.0,NaN,130.42,0.003182,0.009305,39.970257,133.478420,134.334370,130.949620,138.418910,129.50359,126.7100,137.53000,NaN
497,2024-10-28,YUM,134.870,NaN,1.0,NaN,133.04,0.013755,0.006897,50.767624,134.574110,133.394600,134.047180,138.692950,130.99506,129.7100,139.92000,NaN
498,2024-10-28,ZBH,103.550,NaN,1.0,NaN,102.35,0.011725,0.010933,56.671688,107.229390,107.883995,115.284256,107.086190,101.50781,101.7700,115.91237,NaN
499,2024-10-28,ZBRA,363.530,NaN,1.0,NaN,360.09,0.009553,0.009969,43.505260,356.281000,337.452000,311.584440,380.047030,359.12097,320.7700,377.68000,NaN


In [30]:
def prepare_data_v2(main_data: pd.DataFrame) -> pd.DataFrame:
    # Create a copy of the input DataFrame
    df = main_data.copy().reset_index(drop=True)
    
    # Convert the `Date` column to datetime
    df["Date"] = pd.to_datetime(df["Date"])
    
    # Extract year, month, and day from the `Date` column
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    
    # Set the index to `Date` and `Ticker`
    df = df.set_index(["Date", "Ticker"])
    
    return df

In [33]:
prep_data = todays_data.copy()

prep_data = prepare_data_v2(prep_data)

prep_data

Adjusted Close  Today to Tomorrow  Yesterday to Today  \
Date       Ticker                                                          
2024-10-28 A              131.620                NaN                 1.0   
           AAPL           233.490                NaN                 1.0   
           ABBV           189.740                NaN                 1.0   
           ABNB           135.820                NaN                 1.0   
           ABT            114.130                NaN                -1.0   
...                           ...                ...                 ...   
           XYL            130.835                NaN                 1.0   
           YUM            134.870                NaN                 1.0   
           ZBH            103.550                NaN                 1.0   
           ZBRA           363.530                NaN                 1.0   
           ZTS            182.850                NaN                 1.0   

                   Next Day Close  Previous Day Close    Return  Volatility  \
Date       Ticker                                                             
2024-10-28 A                  NaN              130.19  0.010984    0.012264   
           AAPL               NaN              231.41  0.008988    0.013784   
           ABBV               NaN              187.85  0.010061    0.008103   
           ABNB               NaN              134.58  0.009214    0.014882   
           ABT                NaN              114.22 -0.000788    0.010339   
...                           ...                 ...       ...         ...   
           XYL                NaN              130.42  0.003182    0.009305   
           YUM                NaN              133.04  0.013755    0.006897   
           ZBH                NaN              102.35  0.011725    0.010933   
           ZBRA               NaN              360.09  0.009553    0.009969   
           ZTS                NaN              180.01  0.015777    0.013254   

                         RSI      SMA_50     SMA_100     SMA_200  Upper Band  \
Date       Ticker                                                              
2024-10-28 A       21.425411  140.169080  136.640840  137.961560  151.502270   
           AAPL    63.955185  226.950400  222.175250  201.011900  237.876570   
           ABBV    45.739376  192.653440  183.300860  174.407040  196.414630   
           ABNB    57.457336  125.337400  133.320100  143.597980  140.080760   
           ABT     50.131912  114.222595  109.517050  109.521515  119.484985   
...                      ...         ...         ...         ...         ...   
           XYL     39.970257  133.478420  134.334370  130.949620  138.418910   
           YUM     50.767624  134.574110  133.394600  134.047180  138.692950   
           ZBH     56.671688  107.229390  107.883995  115.284256  107.086190   
           ZBRA    43.505260  356.281000  337.452000  311.584440  380.047030   
           ZTS     42.451588  189.078800  183.390370  179.182240  197.458200   

                   Lower Band   Support  Resistance Action  Year  Month  Day  
Date       Ticker                                                             
2024-10-28 A        128.73973  130.1900   148.24400    NaN  2024     10   28  
           AAPL     222.29343  216.3200   236.48000    NaN  2024     10   28  
           ABBV     185.96626  186.5400   197.77069    NaN  2024     10   28  
           ABNB     125.49424  114.2800   137.19000    NaN  2024     10   28  
           ABT      111.09392  110.2504   119.39000    NaN  2024     10   28  
...                       ...       ...         ...    ...   ...    ...  ...  
           XYL      129.50359  126.7100   137.53000    NaN  2024     10   28  
           YUM      130.99506  129.7100   139.92000    NaN  2024     10   28  
           ZBH      101.50781  101.7700   115.91237    NaN  2024     10   28  
           ZBRA     359.12097  320.7700   377.68000    NaN  2024     10   28  
           ZT

In [34]:
# Drop columns that are not needed for the model
select_columns_to_drop = ["Action", "Next Day Close", "Previous Day Close", "Resistance", "Upper Band", "SMA_50", "SMA_200"]

prep_data = prep_data.drop(columns=select_columns_to_drop)

prep_data

Adjusted Close  Today to Tomorrow  Yesterday to Today  \
Date       Ticker                                                          
2024-10-28 A              131.620                NaN                 1.0   
           AAPL           233.490                NaN                 1.0   
           ABBV           189.740                NaN                 1.0   
           ABNB           135.820                NaN                 1.0   
           ABT            114.130                NaN                -1.0   
...                           ...                ...                 ...   
           XYL            130.835                NaN                 1.0   
           YUM            134.870                NaN                 1.0   
           ZBH            103.550                NaN                 1.0   
           ZBRA           363.530                NaN                 1.0   
           ZTS            182.850                NaN                 1.0   

                     Return  Volatility        RSI     SMA_100  Lower Band  \
Date       Ticker                                                            
2024-10-28 A       0.010984    0.012264  21.425411  136.640840   128.73973   
           AAPL    0.008988    0.013784  63.955185  222.175250   222.29343   
           ABBV    0.010061    0.008103  45.739376  183.300860   185.96626   
           ABNB    0.009214    0.014882  57.457336  133.320100   125.49424   
           ABT    -0.000788    0.010339  50.131912  109.517050   111.09392   
...                     ...         ...        ...         ...         ...   
           XYL     0.003182    0.009305  39.970257  134.334370   129.50359   
           YUM     0.013755    0.006897  50.767624  133.394600   130.99506   
           ZBH     0.011725    0.010933  56.671688  107.883995   101.50781   
           ZBRA    0.009553    0.009969  43.505260  337.452000   359.12097   
           ZTS     0.015777    0.013254  42.451588  183.390370   181.45679   

                    Support  Year  Month  Day  
Date       Ticker                              
2024-10-28 A       130.1900  2024     10   28  
           AAPL    216.3200  2024     10   28  
           ABBV    186.5400  2024     10   28  
           ABNB    114.2800  2024     10   28  
           ABT     110.2504  2024     10   28  
...                     ...   ...    ...  ...  
           XYL     126.7100  2024     10   28  
           YUM     129.7100  2024     10   28  
           ZBH     101.7700  2024     10   28  
           ZBRA    320.7700  2024     10   28  
           ZTS     180.0100  2024     10   28  

[501 rows x 12 columns]

In [38]:
X = prep_data.drop(columns=["Today to Tomorrow"])

X_scaler = StandardScaler()

X_scaled = X_scaler.fit_transform(X)

X_scaled


array([[-0.17354956,  0.63822489,  0.44088412, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.02664615,  0.63822489,  0.28147951, ...,  0.        ,
         0.        ,  0.        ],
       [-0.05933169,  0.63822489,  0.36717929, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.22871294,  0.63822489,  0.50004695, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.28220176,  0.63822489,  0.32659604, ...,  0.        ,
         0.        ,  0.        ],
       [-0.07287197,  0.63822489,  0.82376347, ...,  0.        ,
         0.        ,  0.        ]])

In [41]:
import pickle

model_name = "clf_XGB_v1.pkl"
model_path = f"../../models/{model_name}"

with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../models/clf_XGB_v1.pkl'

In [ ]:
# Load model from file

import pickle
# Import the LogisticRegression class from the linear_model module

model_name = "clf_XGB_v1.pkl"
model_path = f"../../models/{model_name}"

with open(model_path, 'rb') as file:
    loaded_model = pickle.load(file)

# Use the loaded model to make predictions
predictions = loaded_model.predict(pivot_data)
